blocco di base trovato, da usare al posto di Word2Vec

from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True) 
#modello utilizzato al posto di Word2Vec

frase = ''
embedding = model.encode(frase)['dense_vecs']

print(embedding)

**PARTE 1 DEL PROGETTO**
***FACCIO IMPORT***

In [2]:
import os 
import fitz #to import pdf
import re
import chromadb #for vectordb
from FlagEmbedding import BGEM3FlagModel
import ollama
import time

**CARICO IL MODELLO DI EMBEDDING CHE HO SCELTO**

In [3]:
print("Caricamento del modello BGE-M3...")
model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)
print("Modello caricato.")

Caricamento del modello BGE-M3...


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

Modello caricato.


**SETTO I PERCORSI DEI FILE**

In [4]:
PDF_PATH = "ETR_500.pdf" 
DB_PATH = "/Users/raffaeleciccarone/Desktop/project_ibm/chroma_db_data"         
COLLECTION_NAME = "manuale_manutenzione_ETR500"

In [4]:
def cleaning(text):
    text = text.lower()
    text = re.sub(r'\n\d+\n', ' ', text) # alone numbers
    text = text.replace('...', '')       # dots
    text = re.sub(r'\s+', ' ', text)     # spaces
    text = re.sub(r"http\S+|www\.\S+", " ", text) # link
    text = re.sub(r"[^\w\s]", " ", text) # punctuation

    return text.strip()

def chunking(text, chunk_size=300, overlap=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

**ESTRAZIONE TESTO E CHUNK**

In [5]:
print(f"Lettura del file: {PDF_PATH}")
doc = fitz.open(PDF_PATH)
full_text = ""

for page in doc:
    full_text += page.get_text()

print("Pulizia del testo...")
clean_text = cleaning(full_text)
print("Testo pulito!")

print("Suddivisione in chunks...")
chunks = chunking(clean_text)

print(f"Creati: {len(chunks)} CHUNKS.")

Lettura del file: ETR_500.pdf
Pulizia del testo...
Testo pulito!
Suddivisione in chunks...
Creati: 148 CHUNKS.


**CREAZIONE CHROMA DB**

In [6]:
client = chromadb.PersistentClient(path=DB_PATH)
collection = client.get_or_create_collection(name=COLLECTION_NAME)

print("Generazione embedding e salvataggio...")

#population of the database
ids = [] 
embeddings = []
documents = []

for i, chunk in enumerate(chunks):
    vector = model.encode(chunk)['dense_vecs'] #model.encode returns a dictionary and we take just the 'dense_vecs' key
    
    ids.append(f"id_{i}")       # just one for every chunk
    embeddings.append(vector)   # vector
    documents.append(chunk)     # original text

# Append to the database
collection.add(
    ids=ids,
    embeddings=embeddings,
    documents=documents
)

print(f"Fatto! Salvati {len(chunks)} vettori nella cartella '{DB_PATH}'.")

Generazione embedding e salvataggio...


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Fatto! Salvati 148 vettori nella cartella '/Users/raffaeleciccarone/Desktop/project_ibm/chroma_db_data'.


**TEST DEL RECUPERO INFO | retrieval**

In [ ]:
query_utente = "Come si esegue la manutenzione del sistema di frenata dell'ETR 500?"
print(f"Domanda: {query_utente}")

query_vector = model.encode(query_utente)['dense_vecs'] #the same as before

results = collection.query(
    query_embeddings=query_vector,
    n_results=3
)

print("\n--- DOCUMENTI RECUPERATI DAL DB ---")
for i, doc in enumerate(results['documents'][0]):
    print(f"\nRISULTATO #{i+1}:")
    print(doc) # print the retrieved documents
    print("-" * 40)

Domanda: Come si esegue la manutenzione del sistema di frenata dell'ETR 500?

--- DOCUMENTI RECUPERATI DAL DB ---

RISULTATO #1:
manuale di manutenzione e riparazione di 1 livello elettrotreno etr 500 plt volume mr1 03c impianto pneumatico e freno carrozze trenitalia s p a divisione passeggeri direzione operativa trazione e manutenzione corrente manutenzione corrente etr e mezzi leggeri elettrotreno etr 500 plt progetto della società frensistemi breda costruzioni ferroviarie manuale di manutenzione e riparazione di 1 livello volume mr1 03c impianto pneumatico e freno carrozze edizione dicembre 2002 trenitalia s p a divisione passeggeri direzione operativa trazione e manutenzione corrente manutenzione corrente etr e mezzi leggeri trenitalia s p a proprietà riservata i 500 plt mr1 c prefazione il presente manuale è destinato al personale tecnico degli impianti di manutenzione ro tabili il manuale contiene tutte le informazioni tecniche ad uso del personale necessarie per una corretta man

In [5]:
!ollama pull llama3
!ollama pull llama3.2
!ollama pull granite3-dense:2b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling 6a0746a1ec1a: 100% ▕██████████████████▏ 4.7 GB                         
pulling 4fa551d4f938: 100% ▕██████████████████▏  12 KB                         
pulling 8ab4849b038c: 100% ▕██████████████████▏  254 B                         
pulling 577073ffcc6c: 100% ▕██████████████████▏  110 B                         
pulling 3f8eb4da87fa: 100% ▕██████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████

**TEST EFFICIENZA IN TEMPO**

In [ ]:
query_text = input('Fai la tua domanda sulla manutenzione del treno:\n\n')
model_list = ['llama3', 'llama3.2', 'granite3-dense:2b']

def final_test(query_text, model, DB_PATH, COLLECTION_NAME, model_list):
    print(f"Il tuo aiutante manutentore sta pensando: '{query_text}'...")

    #retrieve from vectordb
    query_vec = model.encode(query_text)['dense_vecs']

    client = chromadb.PersistentClient(path=DB_PATH)
    collection = client.get_or_create_collection(name=COLLECTION_NAME)
    results = collection.query(query_embeddings=query_vec, n_results=3)

    # Join texts in a single string
    context_list = results['documents'][0]
    context_text = "\n---\n".join(context_list)

    #define model prompt with summary of all retrieved chunks plus user query
    prompt_template = f"""
    Sei un assistente esperto e specializzato nella manutenzione ferroviaria del treno ETR 500.
    Rispondi alla domanda che ti viene posta prendendo in considerazione e basandoti ESCLUSIVAMENTE sul contesto fornito.
    Se l'informazione non c'è nel contesto, rispondi che non hai la risposta. NON inventare procedure.

    CONTESTO ESTRATTO DAL MANUALE:
    {context_text}

    DOMANDA UTENTE:
    {query_text}

    RISPOSTA (in italiano tecnico e preciso):
    """
    for m in model_list:
        print(f"\n\n=== TEST DEL MODELLO: {m} ===")
        #GOOOOOOOOOOOOOOO!!!
        start_time = time.time()

        response = ollama.chat(model=m, messages=[
            {'role': 'user', 'content': prompt_template},
        ])

        #stop and print...
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f'Tempo per inferenza: {elapsed_time:.2f} secondi')

        print("\n" + "=D "*50)
        print(f"RISPOSTA GENERATA DA '{m}':")
        print("=P "*50)
        print(response['message']['content'])

final_test(query_text, model, DB_PATH, COLLECTION_NAME, model_list)

Il tuo aiutante manutentore sta pensando: 'come fare prova freno? dividi il processo in tre parti'...


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.




=== TEST DEL MODELLO: llama3 ===
Tempo per inferenza: 122.29 secondi

=D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D 
RISPOSTA GENERATA DA 'llama3':
=P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P 
La procedura di prova del freno ETR 500 è regolata dalla normativa utmr prot 0311 del 20/04/2001. La prova del freno consiste in una serie di prove mirate a verificarne eventuali anomalie.

**Parte 1: Configurazione della prova**

* Utilizzare le apparecchiature di prova di tipo ACPF di costruzione di Trenitalia.
* Progettare la configurazione della prova del freno in base alle specifiche delle carrozze ETR 500.

**Parte 2: Esecuzione della prova**

* Effettuare una serie di prove all'impianto frenante della carrozza, come previsto dalla normativa.
* Utilizzare le apparecchiature di prova